In [1]:
import os,sys,string, time
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import xgboost
from xgboost import XGBClassifier
import joblib
import seaborn as sns
from importlib import reload
import pickle

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')


Success


## Reading in files

In [2]:
Params = {"Run":"run1", #The run number, so far either "run1" or "run3"
          "Load_single_file":False, #This will override everything else, put the desired file in the "single_file" line
          "single_file":10,
          "Load_standard":True,
          "Load_DetVars":False,
          "Only_keep_common_DetVar_evs":True,
          "Load_data":False,
          "FLATTEN":True, #Have one row per reconstructed object in the analysis dataframe
          "only_presel":False, #Create small files containing only variables necessary for pre-selection, for making pre-selection plots
          "EXT_in_training":True,
          "Load_pi0_signal":False} #Otherwise loads e+e- samples by default

loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/my_vars/"

In [3]:
signal_samples_dict = {}
# end_string = "_FINAL"
end_string = "_full_Finished"

Presel_overlay = pd.read_pickle(loc_pkls+"Preselected_overlay_"+Params["Run"]+f"_flattened{end_string}.pkl")

#Always load in EXT for pre-selection, even though might not be used in the training
Presel_EXT = pd.read_pickle(loc_pkls+"Preselected_beamoff_"+Params["Run"]+f"_flattened{end_string}.pkl")
Presel_dirt = pd.read_pickle(loc_pkls+"Preselected_dirtoverlay_"+Params["Run"]+f"_flattened{end_string}.pkl")

start_load_str = loc_pkls
if Params["Load_pi0_signal"] == False: HNL_samples = Constants.HNL_ee_samples_names
if Params["Load_pi0_signal"] == True: 
    start_load_str += f"pi0_selection/"
    HNL_samples = Constants.HNL_mass_pi0_samples_names
if Params["Load_single_file"] == True: HNL_samples = [Params["single_file"]]
    
for HNL_mass in HNL_samples:
    Presel_signal = pd.read_pickle(start_load_str+f"Preselected_{HNL_mass}_"+Params["Run"]+f"_flattened{end_string}.pkl")
    signal_samples_dict[HNL_mass] = Presel_signal

print(Presel_overlay.keys())

Index(['run', 'sub', 'evt', 'nslice', 'n_pfps', 'n_tracks', 'n_showers',
       'swtrig_pre', 'swtrig_post', 'trk_sce_start_x_v', 'trk_sce_start_y_v',
       'trk_sce_start_z_v', 'trk_sce_end_x_v', 'trk_sce_end_y_v',
       'trk_sce_end_z_v', 'shr_theta_v', 'shr_phi_v', 'shr_px_v', 'shr_py_v',
       'shr_pz_v', 'shrclusdir0', 'shrclusdir1', 'shrclusdir2',
       'shr_energy_tot', 'trk_theta_v', 'trk_phi_v', 'trk_dir_x_v',
       'trk_dir_y_v', 'trk_dir_z_v', 'trk_energy', 'trk_energy_hits_tot',
       'trk_energy_tot', 'trk_score_v', 'trk_calo_energy_u_v', 'trk_end_x_v',
       'trk_chipr_best', 'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y',
       'NeutrinoEnergy2', 'SliceCaloEnergy2', 'nu_flashmatch_score',
       'contained_sps_ratio', 'flash_time', 'contained_fraction', 'trk_score',
       'crtveto', 'shr_tkfit_dedx_U', 'shr_tkfit_dedx_V', 'shr_tkfit_dedx_Y',
       'shr_tkfit_dedx_max', 'shr_tkfit_2cm_dedx_Y', 'shr_chipr',
       'trk_bragg_p', 'trk_bragg_p_v', 'trk_chipr'

## Splitting into test and training samples

In [4]:
#overlay_train_frac, signal_train_frac, EXT_train_frac
overlay_train, overlay_test, EXT_train, EXT_test, signal_train_dict, signal_test_dict = Functions.Split_samples(Presel_overlay, signal_samples_dict, 
                                                                                                      Presel_EXT, 0.5, 0.5, 0.3)


Length overlay train 23028
Length overlay test 23028
Length EXT train 1987
Length EXT test 4639
Length 2_ee train 7413
Length 2_ee test 7413
Length 10_ee train 6297
Length 10_ee test 6297
Length 20_ee train 6950
Length 20_ee test 6951
Length 50_ee train 6351
Length 50_ee test 6352
Length 100_ee train 6606
Length 100_ee test 6607
Length 150_ee train 6376
Length 150_ee test 6376


In [5]:
ultimate_feature_list = ['n_pfps', 'n_tracks', 'shr_theta_v', 'shr_phi_v', 'shr_pz_v', 'shrclusdir2', 'shr_energy_tot',
                         'trk_theta_v', 'trk_phi_v','trk_dir_z_v', 'trk_energy', 'trk_energy_tot', 'trk_score_v',
                         'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y', 'NeutrinoEnergy2']#, 'nu_flashmatch_score']

ultimate_feature_list += ['shr_tkfit_dedx_max', 'topological_score']
if Params["Run"]=="run3":
    ultimate_feature_list += ['nu_flashmatch_score']
print(f"Number of features is {len(ultimate_feature_list)}")

bdt_vars = ultimate_feature_list #This is using just the most important variables list

if Params["Load_pi0_signal"] == False: BDT_name = f"{end_string}"
if Params["Load_pi0_signal"] == True: BDT_name = f"{end_string}"
if Params["EXT_in_training"] == True: BDT_name = f"{end_string}"

BDT_name = "_full_Finished_3"

var_list = bdt_vars

print(f"Saving variables used as {BDT_name}_"+Params["Run"]+".pkl")
if Params["Load_pi0_signal"] == False:
    with open(f"bdts/input_vars/{BDT_name}_"+Params["Run"], "wb") as fp:   #Pickling
        pickle.dump(var_list, fp)
elif Params["Load_pi0_signal"] == True:
    with open(f"bdts/pi0_selection/input_vars/{BDT_name}_"+Params["Run"], "wb") as fp:   #Pickling
        pickle.dump(var_list, fp)
        
signal_samples_dict.keys()

Number of features is 19
Saving variables used as _full_Finished_3_run1.pkl


dict_keys(['2_ee', '10_ee', '20_ee', '50_ee', '100_ee', '150_ee'])

In [6]:
save_pkls = input("Do you want to save the test sample pkls? y/n ")

if save_pkls == "y":
    Functions.Save_test_pkls(Params, loc_pkls, "_full_Finished_new", overlay_test, signal_test_dict, EXT_test)
else: print("Not saved .pkls")

Do you want to save the test sample pkls? y/n  n


Not saved .pkls


In [7]:
combined_dict, labels_dict, bkg_train = Functions.Make_train_labels_and_dicts(Params, bdt_vars, overlay_train, EXT_train, signal_train_dict)

if Params["EXT_in_training"] == True:
    print("Adding overlay, EXT and dirt into test set")
    bkg_test = pd.concat([overlay_test,EXT_test, Presel_dirt])
else: bkg_test = overlay_test.copy()

Adding overlay, EXT and dirt into test set


In [8]:
reload(Functions)   
def Test_vs_train_plots(test_sig, test_bkg, train_sig, train_bkg, bdt_dict, bins_dict):
    """
    Plotting the BDT score results for test and training sets to check for overtraining.
    """
    test_results_sig_dict, test_results_bkg_dict = {}, {}
    train_results_sig_dict, train_results_bkg_dict = {}, {}
    
    for HNL_mass in test_sig:
        results_sig = Functions.logit(bdt_dict[HNL_mass].predict(test_sig[HNL_mass]))
        results_bkg = Functions.logit(bdt_dict[HNL_mass].predict(test_bkg))

        train_results_sig = Functions.logit(bdt_dict[HNL_mass].predict(train_sig[HNL_mass]))
        train_results_bkg = Functions.logit(bdt_dict[HNL_mass].predict(train_bkg))

        test_results_sig_dict.update({HNL_mass:results_sig})
        test_results_bkg_dict.update({HNL_mass:results_bkg})

        train_results_sig_dict.update({HNL_mass:train_results_sig})
        train_results_bkg_dict.update({HNL_mass:train_results_bkg})
    
    # hist_range=[0,1.0] #For when NOT using logit(results)
    # n_bins=20

    for HNL_mass in test_sig:
        plt.figure(figsize=(10,7))
        # plt.hist(train_results_sig_dict[HNL_mass],bins=n_bins, range=hist_range, density=True,alpha=0.4,color='red',label=f'Train {HNL_mass}MeV HNL' )
        plt.hist(train_results_sig_dict[HNL_mass],bins=bins_dict[HNL_mass], density=True,alpha=0.4,color='red',label=f'Train {HNL_mass} MeV HNL' )
        counts,bin_edges = np.histogram(test_results_sig_dict[HNL_mass],bins=bins_dict[HNL_mass],density=True)
        bin_centers = (bin_edges[:-1] + bin_edges[1:])/2.
        plt.plot(bin_centers,counts,marker='o',linestyle="None",color='red',label=f'Test {HNL_mass} MeV HNL')

        # plt.hist(train_results_bkg_dict[HNL_mass], bins = n_bins, range = hist_range, density = True, alpha = 0.4, color = 'orange', label = r'Train overlay')
        plt.hist(train_results_bkg_dict[HNL_mass], bins=bins_dict[HNL_mass], density = True, alpha = 0.4, color = 'orange', label = r'Train bkg')
        counts,bin_edges = np.histogram(test_results_bkg_dict[HNL_mass],bins=bins_dict[HNL_mass],density = True)
        bin_centers = (bin_edges[:-1] +  bin_edges[1:])/2.
        plt.plot(bin_centers,counts,marker='o',linestyle ="None",color='orange',label = r'Test bkg')
        plt.legend()
        

In [9]:
xgb_train_dict, xgb_sig_train_dict, xgb_sig_test_dict, xgb_train_bkg, xgb_test_bkg = Functions.Prepare_for_xgb(Params, bdt_vars, combined_dict, 
                                                                                                     signal_train_dict, bkg_train,  signal_test_dict, 
                                                                                                     bkg_test, labels_dict, missing=-9999.0)

In [10]:
combined_test_dict, labels_test_dict={}, {}
for HNL_mass in xgb_sig_train_dict:
    combined_test_dict[HNL_mass] = pd.concat([signal_test_dict[HNL_mass][bdt_vars], bkg_test[bdt_vars]])
    labels_test_dict[HNL_mass] = [1]*len(signal_test_dict[HNL_mass]) + [0]*len(bkg_test)

xgb_combined_test_dict = {}
for HNL_mass in xgb_sig_train_dict:
    xgb_combined_test_dict[HNL_mass] = xgboost.DMatrix(combined_test_dict[HNL_mass][bdt_vars], label=labels_test_dict[HNL_mass], 
                                                   missing=-9999.0, feature_names=bdt_vars)

In [11]:
xgb_param = {'booster': 'dart',
             'max_depth':6,
             'eta':0.3,
             'objective':'binary:logistic',
             # 'eval_metric':'auc', #area under ROC curve
             'eval_metric':'logloss',
             # 'eval_metric':'mae',
             'tree_method':'hist',
             'rate_drop': 0.1,
             'skip_drop': 0.3,
#        'subsample':0.5,
             'scale_pos_weight': float(len(bkg_train))/float(len(signal_train_dict[HNL_mass]))
            }
progress = dict()

print(f"BDT name is {BDT_name}")

BDT name is _full_Finished_3


In [12]:
train_BDTs = input(f"Do you want to train new BDT models with names {BDT_name}? y/n ")

if train_BDTs == "y":
    Functions.Train_BDTs(Params, bdt_vars, BDT_name, xgb_train_dict, xgb_sig_test_dict, xgb_test_bkg, xgb_param, xgb_combined_test_dict, 
               progress, num_round = 150, early_stop=20)
    print("Finished all!")
else: print("Not retraining BDTs")

Do you want to train new BDT models with names _full_Finished_3? y/n  y


Training 2_ee BDT

Training 10_ee BDT

Training 20_ee BDT

Training 50_ee BDT

Training 100_ee BDT

Training 150_ee BDT

Finished all!


## Optimizing

In [ ]:
optimized params = {'booster': 'dart', 'max_depth': 7, 'eta': 0.3, 'objective': 'binary:logistic', 'eval_metric': 'logloss',
                    'tree_method': 'hist', 'scale_pos_weight': 3.923306148055207,
                    'min_child_weight': 1, 'subsample': 1.0, 'colsample_bytree': 1.0, 'rate_drop': 0.1, 'skip_drop': 0.3}


In [ ]:
#OPTIMIZING

HNL_mass = "100_ee"

old_params = {'booster': 'dart',
             'max_depth':6,
             'eta':0.3,
             'objective':'binary:logistic',
             'tree_method':'hist',
             'rate_drop': 0.1,
             'skip_drop': 0.5#,
#        'subsample':0.5,
#        'scale_pos_weight': float(len(data_bkg_train))/float(len(data_sig_train)),
            }

old_watchlist = [(xgb_train_dict[HNL_mass], 'train'), (xgb_sig_test_dict[HNL_mass], 'test_sig'), (xgb_test_bkg,'test_bkg')]

cv_results = xgboost.cv(
    xgb_param,
    xgb_train_dict[HNL_mass],
    num_boost_round=100,
    seed=42,
    nfold=5,
    metrics={'auc'},
    early_stopping_rounds=5
)

In [ ]:
cv_results

print(cv_results["test-auc-mean"].max())
print(cv_results["test-auc-mean"].argmax())

In [ ]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(4,10)
    for min_child_weight in range(1,6)
]

In [ ]:
# Define initial best params and MAE
max_auc = 0.0
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))    # Update our parameters
    xgb_param['max_depth'] = max_depth
    xgb_param['min_child_weight'] = min_child_weight    # Run CV
    cv_results = xgboost.cv(
        xgb_param,
        xgb_train_dict[HNL_mass],
        num_boost_round=100,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=10
    )    # Update best eval metric
    mean_auc = cv_results["test-auc-mean"].max()
    boost_rounds = cv_results["test-auc-mean"].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
    if mean_auc > max_auc:
        max_auc = mean_auc
        best_params = (max_depth, min_child_weight)
    
print("Best params: {}, {}, AUC: {}".format(best_params[0], best_params[1], max_auc))

In [ ]:
cv_results = xgboost.cv(
    xgb_param,
    xgb_train_dict[HNL_mass],
    num_boost_round=100,
    seed=42,
    nfold=5,
    metrics={'logloss'},
    early_stopping_rounds=5
)

cv_results

print(cv_results["test-logloss-mean"].min())
print(cv_results["test-logloss-mean"].argmin())

In [ ]:
# Define initial best params and MAE
min_logloss = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))    # Update our parameters
    xgb_param['max_depth'] = max_depth
    xgb_param['min_child_weight'] = min_child_weight    # Run CV
    cv_results = xgboost.cv(
        xgb_param,
        xgb_train_dict[HNL_mass],
        num_boost_round=100,
        seed=42,
        nfold=5,
        metrics={'logloss'},
        early_stopping_rounds=10
    )    # Update best eval metric
    mean_logloss = cv_results["test-logloss-mean"].min()
    boost_rounds = cv_results["test-logloss-mean"].argmin()
    print("\tlogloss {} for {} rounds".format(mean_logloss, boost_rounds))
    if mean_logloss < min_logloss:
        min_logloss = mean_logloss
        best_params = (max_depth, min_child_weight)
    
print("Best params: {}, {}, logloss: {}".format(best_params[0], best_params[1], min_logloss))

In [ ]:
xgb_param['max_depth'] = 7
xgb_param['min_child_weight'] = 1
xgb_param['eval_metric'] = "logloss"

gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

print(gridsearch_params)
print(xgb_param)

In [ ]:
min_logloss = float("Inf")
best_params = None
for subsample, colsample in gridsearch_params:
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))    # Update our parameters
    xgb_param['subsample'] = subsample
    xgb_param['colsample_bytree'] = colsample    # Run CV
    cv_results = xgboost.cv(
        xgb_param,
        xgb_train_dict[HNL_mass],
        num_boost_round=100,
        seed=42,
        nfold=5,
        metrics={'logloss'},
        early_stopping_rounds=10
    )    # Update best eval metric
    mean_logloss = cv_results["test-logloss-mean"].min()
    boost_rounds = cv_results["test-logloss-mean"].argmin()
    print("\tlogloss {} for {} rounds".format(mean_logloss, boost_rounds))
    if mean_logloss < min_logloss:
        min_logloss = mean_logloss
        best_params = (subsample, rate_drop)
    
print("Best params: {}, {}, logloss: {}".format(best_params[0], best_params[1], min_logloss))

In [ ]:
xgb_param['subsample'] = 1.0
xgb_param['colsample_bytree'] = 1.0

print(xgb_param)



In [ ]:
#0.20378282252634553 for 0.0 rate drop and skip drop

gridsearch_params = [
    (rate_drop, skip_drop)
    for rate_drop in [i/10. for i in range(1,8)]
    for skip_drop in [i/10. for i in range(1,11)]
]

print(gridsearch_params)


In [ ]:
min_logloss = float("Inf")
best_params = None
for rate_drop, skip_drop in gridsearch_params:
    print("CV with rate_drop={}, skip_drop={}".format(
                             rate_drop,
                             skip_drop))    # Update our parameters
    xgb_param['rate_drop'] = rate_drop
    xgb_param['skip_drop'] = skip_drop    # Run CV
    cv_results = xgboost.cv(
        xgb_param,
        xgb_train_dict[HNL_mass],
        num_boost_round=100,
        seed=42,
        nfold=5,
        metrics={'logloss'},
        early_stopping_rounds=10
    )    # Update best eval metric
    mean_logloss = cv_results["test-logloss-mean"].min()
    boost_rounds = cv_results["test-logloss-mean"].argmin()
    print("\tlogloss {} for {} rounds".format(mean_logloss, boost_rounds))
    if mean_logloss < min_logloss:
        min_logloss = mean_logloss
        best_params = (rate_drop, skip_drop)
    
print("Best params: {}, {}, logloss: {}".format(best_params[0], best_params[1], min_logloss))

In [ ]:
xgb_param['rate_drop'] = 0.1
xgb_param['skip_drop'] = 0.3

print(xgb_param)

In [ ]:

min_logloss = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))    # We update our parameters
    xgb_param['eta'] = eta    # Run and time CV
    cv_results = xgboost.cv(
            xgb_param,
            xgb_train_dict[HNL_mass],
            num_boost_round=300,
            seed=42,
            nfold=5,
            metrics=['logloss'],
            early_stopping_rounds=10
            )    # Update best score
    mean_logloss = cv_results["test-logloss-mean"].min()
    boost_rounds = cv_results["test-logloss-mean"].argmin()
    print("\tlogloss {} for {} rounds\n".format(mean_logloss, boost_rounds))
    if mean_logloss < min_logloss:
        min_logloss = mean_logloss
        best_params = eta
        
print("Best params: {}, logloss: {}".format(best_params, min_logloss))

In [ ]:
xgb_param['eta'] = 0.3

print(xgb_param)

## Test vs. train results

In [ ]:
print(f"Loading BDT models named {BDT_name}")

sample_names = xgb_train_dict.keys()
bdt_model_dict = {}
sig_results_test_dict, bkg_results_test_dict = {}, {}
sig_results_train_dict, bkg_results_train_dict = {}, {}

max_sig_results, max_bkg_results = {}, {}
min_sig_results, min_bkg_results = {}, {}

bins_dict={}

if Params["Load_pi0_signal"]==False: sample_type = "ee"
if Params["Load_pi0_signal"]==True: sample_type = "pi0"

for HNL_mass in sample_names:
    bdt = xgboost.Booster()
    if HNL_mass.split("_")[1] == "pi0": loc = "bdts/pi0_selection/"
    if HNL_mass.split("_")[1] == "ee": loc = "bdts/"
    bdt.load_model(loc+Params["Run"]+f"_{HNL_mass}{BDT_name}.json")
    
    bdt_model_dict[HNL_mass] = bdt
    
    sig_results_test_dict[HNL_mass] = Functions.logit(bdt.predict(xgb_sig_test_dict[HNL_mass]))
    bkg_results_test_dict[HNL_mass] = Functions.logit(bdt.predict(xgb_test_bkg))
    
    sig_results_train_dict[HNL_mass] = Functions.logit(bdt.predict(xgb_sig_train_dict[HNL_mass]))
    bkg_results_train_dict[HNL_mass] = Functions.logit(bdt.predict(xgb_train_bkg))
    
    max_sig_results[HNL_mass] = max([max(sig_results_test_dict[HNL_mass]), max(sig_results_train_dict[HNL_mass])])
    max_bkg_results[HNL_mass] = max([max(bkg_results_test_dict[HNL_mass]), max(bkg_results_train_dict[HNL_mass])])
    
    min_sig_results[HNL_mass] = min([min(sig_results_test_dict[HNL_mass]), min(sig_results_train_dict[HNL_mass])])
    min_bkg_results[HNL_mass] = min([min(bkg_results_test_dict[HNL_mass]), min(bkg_results_train_dict[HNL_mass])])
    
    low_edge_sig, low_edge_bkg = np.floor(min_sig_results[HNL_mass]), np.floor(min_bkg_results[HNL_mass])
    high_edge_sig, high_edge_bkg = np.ceil(max_sig_results[HNL_mass]), np.ceil(max_bkg_results[HNL_mass])

    bins_dict[HNL_mass] = np.arange(low_edge_bkg, high_edge_sig)
    
    
print(bdt_model_dict.keys())



In [ ]:
reload(Functions)
sample_norms = Functions.Get_sample_norms(Params, sample_names, 0.5, 0.5, 0.3)

print(sample_norms)

xgb_overlay = xgboost.DMatrix(overlay_test[bdt_vars], label=[0]*len(overlay_test), missing=-9999.0, feature_names=bdt_vars)
xgb_beamoff = xgboost.DMatrix(EXT_test[bdt_vars], label=[0]*len(EXT_test), missing=-9999.0, feature_names=bdt_vars)
xgb_dirt = xgboost.DMatrix(Presel_dirt[bdt_vars], label=[0]*len(Presel_dirt), missing=-9999.0, feature_names=bdt_vars)


In [ ]:
Test_vs_train_plots(xgb_sig_test_dict, xgb_test_bkg, xgb_sig_train_dict, xgb_train_bkg,bdt_model_dict, bins_dict)

## ROC curves

In [ ]:
sample_names = xgb_train_dict.keys()
bdt_model_dict = {}
sig_results_test_dict, bkg_results_test_dict = {}, {}
sig_results_train_dict, bkg_results_train_dict = {}, {}
overlay_test_results, beamoff_test_results, dirt_test_results = {}, {}, {}

if Params["Load_pi0_signal"]==False: sample_type = "ee"
if Params["Load_pi0_signal"]==True: sample_type = "pi0"

# xgb_overlay 
# xgb_beamoff 
# xgb_dirt

for HNL_mass in sample_names:
    bdt = xgboost.Booster()
    if HNL_mass.split("_")[1] == "pi0": loc = "bdts/pi0_selection/"
    if HNL_mass.split("_")[1] == "ee": loc = "bdts/"
    bdt.load_model(loc+Params["Run"]+f"_{HNL_mass}{BDT_name}.json")
    
    bdt_model_dict[HNL_mass] = bdt
    
    sig_results_test_dict[HNL_mass] = bdt.predict(xgb_sig_test_dict[HNL_mass])
    bkg_results_test_dict[HNL_mass] = bdt.predict(xgb_test_bkg)
    
    sig_results_train_dict[HNL_mass] = bdt.predict(xgb_sig_train_dict[HNL_mass])
    bkg_results_train_dict[HNL_mass] = bdt.predict(xgb_train_bkg)
    
    overlay_test_results[HNL_mass] = bdt.predict(xgb_overlay)
    beamoff_test_results[HNL_mass] = bdt.predict(xgb_beamoff)
    dirt_test_results[HNL_mass] = bdt.predict(xgb_dirt)
    
print(bdt_model_dict.keys())

In [ ]:
def RocCurves_test(test_sig,test_bkg,mass_label):
    """
    For plotting ROC curves of test samples only.
    """
    
    def PosRate(cut_val,var):
        return len(var[var>cut_val])/len(var)
    
    def NegRate(cut_val,var):
        return len(var[var<cut_val])/len(var)
    
    def CalcRate(sig,bkg):
        cuts=np.linspace(0,1,200)
        FPs=[]
        TPs=[]


        for cut in cuts:
            FPs.append(NegRate(cut,bkg))
            TPs.append(PosRate(cut,sig))

        FPs=np.array(FPs)
        TPs=np.array(TPs)

        AUC=-1.0*sum(((TPs[:-1]+TPs[1:])/2)*(FPs[:-1]-FPs[1:]))
        rounded_AUC = round(AUC,3)

        plt.plot(FPs,TPs,label=f'{mass_label} MeV, AUC={AUC:.{3}g}')
        # plt.plot(FPs,TPs,label=f'{mass_label} MeV, AUC={rounded_AUC}')
        
    CalcRate(test_sig,test_bkg)
    
def RocCurves_normalised(test_sig, sample_norms, test_overlay, test_beamoff, test_dirt, mass_label):
    """
    For plotting ROC curves of test samples only. 
    Correctly accounted for POT normalisation but not individual weights (effect minimal).
    """
    
    def PosRate(cut_val,var): #Fraction of signal events above cut_val
        return len(var[var>cut_val])/len(var)
    
    def NegRate(cut_val,overlay,beamoff,dirt): #Fraction of all bkg events above cut_val (normalised, not weighted)
        num=(len(overlay[overlay<cut_val])*sample_norms["overlay"])+(len(beamoff[beamoff<cut_val])*sample_norms["beamoff"])+(len(dirt[dirt<cut_val])*sample_norms["dirtoverlay"])
        dom=(len(overlay)*sample_norms["overlay"])+(len(beamoff)*sample_norms["beamoff"])+(len(dirt)*sample_norms["dirtoverlay"])
        return num/dom
    
    def CalcRate(sig,overlay,beamoff,dirt):
        cuts=np.linspace(0,1,50)
        FPs=[]
        TPs=[]
        
        for cut in cuts:
            FPs.append(NegRate(cut,overlay,beamoff,dirt))
            TPs.append(PosRate(cut,sig))

        FPs=np.array(FPs)
        TPs=np.array(TPs)

        AUC=-1.0*sum(((TPs[:-1]+TPs[1:])/2)*(FPs[:-1]-FPs[1:]))
        rounded_AUC = round(AUC,3)

        plt.plot(FPs,TPs,label=f'{mass_label} MeV, AUC={AUC:.{3}g}')
        
    CalcRate(test_sig, test_overlay, test_beamoff, test_dirt)
    
def RocCurves_test_train(test_sig,test_bkg,train_sig,train_bkg, mass_label):
    """
    For plotting ROC curves of test samples only.
    """
    
    def PosRate(cut_val,var):
        return len(var[var>cut_val])/len(var)
    
    def NegRate(cut_val,var):
        return len(var[var<cut_val])/len(var)
    
    def CalcRate(sig,bkg,linestyle):
        cuts=np.linspace(0,1,200)
        FPs=[]
        TPs=[]


        for cut in cuts:
            FPs.append(NegRate(cut,bkg))
            TPs.append(PosRate(cut,sig))

        FPs=np.array(FPs)
        TPs=np.array(TPs)


        AUC=-1.0*sum(((TPs[:-1]+TPs[1:])/2)*(FPs[:-1]-FPs[1:]))
        rounded_AUC = round(AUC,3)
        
        plt.plot(FPs,TPs,label=f'{mass_label} MeV, AUC={AUC:.{3}g}',linestyle=linestyle)
        # plt.plot(FPs,TPs,label=f'{mass_label} MeV, AUC={rounded_AUC}')
        plt.xlabel("False Positives")
        plt.ylabel("True Positives")
        plt.legend()
        
        
    CalcRate(test_sig,test_bkg,linestyle="solid")
    CalcRate(train_sig,train_bkg,linestyle="dashed")


In [ ]:
plt.figure(figsize=[8,8])

save_fig = input("Do you want to save the Figure? y/n ")

for HNL_mass in sig_results_test_dict:
    mass_label = HNL_mass.split("_")[0]
    if mass_label == "2": continue
    # RocCurves_test(sig_results_test_dict[HNL_mass],bkg_results_test_dict[HNL_mass], mass_label)
    RocCurves_normalised(sig_results_test_dict[HNL_mass], sample_norms, overlay_test_results[HNL_mass], beamoff_test_results[HNL_mass], 
                         dirt_test_results[HNL_mass], mass_label)
    
x_flat_line=np.linspace(0,1,50)
y_flat_line = []
for i, x in enumerate(x_flat_line):
    y_flat_line.append(1-x)
    
plt.plot(x_flat_line, y_flat_line, linestyle="dashed", color="black")
    
plt.xlabel("Signal Efficiency")
plt.ylabel("Background Rejection")
plt.legend(fontsize=19)

if save_fig == "y":
    plt.savefig(f"plots/BDT_output/BDT_performance/Test_set_normalised_ROC_" + Params["Run"] + f"_{sample_type}{BDT_name}.png")
    plt.savefig(f"plots/BDT_output/BDT_performance/Test_set_normalised_ROC_" + Params["Run"] + f"_{sample_type}{BDT_name}.pdf")


In [ ]:
plt.figure(figsize=[8,8])

save_fig = input("Do you want to save the Figure? y/n ")

for HNL_mass in sig_results_test_dict:
    mass_label = HNL_mass.split("_")[0]
    if mass_label == "2": continue
    RocCurves_test_train(sig_results_test_dict[HNL_mass],bkg_results_test_dict[HNL_mass],
                         sig_results_train_dict[HNL_mass],bkg_results_train_dict[HNL_mass], mass_label)
    
x_flat_line=np.linspace(0,1,200)
y_flat_line = []
for i, x in enumerate(x_flat_line):
    y_flat_line.append(1-x)
    
plt.plot(x_flat_line, y_flat_line, linestyle="dashed", color="black")
    
plt.xlabel("False Positives")
plt.ylabel("True Positives")
plt.legend(fontsize=19)

if save_fig == "y":
    plt.savefig(f"plots/BDT_output/BDT_performance/Test_and_train_ROC_" + Params["Run"] + f"_{sample_type}{BDT_name}.png")
    plt.savefig(f"plots/BDT_output/BDT_performance/Test_and_train_ROC_" + Params["Run"] + f"_{sample_type}{BDT_name}.pdf")

In [ ]:
var_list = bdt_vars

# BDT_name = "New_20_variables_FIXED"
if Params["Load_pi0_signal"] == False:
    with open(f"bdts/input_vars/{BDT_name}_"+Params["Run"], "wb") as fp:   #Pickling
        pickle.dump(var_list, fp)
elif Params["Load_pi0_signal"] == True:
    with open(f"bdts/pi0_selection/input_vars/{BDT_name}_"+Params["Run"], "wb") as fp:   #Pickling
        pickle.dump(var_list, fp)

# with open("bdts/input_vars/"+BDT_name, "rb") as fp:   # Unpickling
#     b = pickle.load(fp)
    
# print(b)

# Finished Training

## Checking variable correlations

In [ ]:
# bdt_vars = feature_names
HNL_mass = "150_pi0"

In [ ]:
#Taken from Luis' code
# for HNL_mass in HNL_masses:
method = 'kendall'
correlations = signal_samples_dict[HNL_mass][bdt_vars].astype(np.float64).corr(method=method)
plt.figure(figsize=(15,12))
sns.heatmap(correlations,vmin=-1,annot=False,square=True,cbar_kws={'label':method+' correlation'},cmap = 'RdBu_r')
plt.title('Input Variable Correlations')
plt.show()

In [ ]:
#Just looking at most correlated 
corr=signal_samples_dict[HNL_mass][bdt_vars].corr()
high_corr_var=np.where(corr>0.999)
high_corr_var=[(corr.columns[x],corr.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
#high_corr_var
print("Done")

## Looking at feature importances

In [ ]:
a = "150_ee"
print(a.split("_")[1])

print(plt.rcParams['axes.prop_cycle'].by_key()['color'])


In [ ]:
#Make an importances dict   
def Top_N_vars(bdt_model, N_vars):
    importance = bdt.get_score(importance_type="gain")
    # for key in importance.keys():
    #     importance[key] = round(importance[key],1)
    sorted_importance = dict(sorted(importance.items(), key=lambda item: item[1]))
    sorted_importance_list = list(sorted_importance.values())
    sorted_importance_keys= list(sorted_importance.keys())
    top_N = sorted_importance_keys[-N_vars:]
    
    return top_N

def Sorted_importance(bdt_model):
    importance = bdt.get_score(importance_type="gain")
    return importance

top_N_dict = {}
importance_dict = {}
list_of_lists = []
# BDT_name = "ee_Finished"
# BDT_name = "pi0"

# if Params["Load_pi0_signal"] == False: 
#     sample_names = Constants.HNL_ee_samples_names
#     loc = "bdts/"
# if Params["Load_pi0_signal"] == True: 
#     sample_names = Constants.HNL_mass_pi0_samples_names #I should load BOTH
#     loc = "bdts/pi0_selection/"
sample_names = Constants.HNL_ee_samples_names #+ Constants.HNL_mass_pi0_samples_names
# for HNL_mass in Constants.HNL_mass_samples:
for HNL_mass in sample_names:
    bdt = xgboost.Booster()
    if HNL_mass.split("_")[1] == "pi0": loc = "bdts/pi0_selection/"
    if HNL_mass.split("_")[1] == "ee": loc = "bdts/"
    # bdt.load_model("bdts/"+Params["Run"]+f"_{HNL_mass}MeV_ultimate.json")
    # bdt.load_model("bdts/"+Params["Run"]+f"_{HNL_mass}MeV_{BDT_name}.json")
    bdt.load_model(loc+Params["Run"]+f"_{HNL_mass}{BDT_name}.json")
    # print("Number of entries in top20 is " + str(len(Top_N_vars(bdt, 20))))
    # top_N_dict[HNL_mass] = Top_N_vars(bdt, 20)
    top_N = Top_N_vars(bdt, 50)
    top_N_dict[HNL_mass] = Top_N_vars(bdt, 50)
    list_of_lists.append(Top_N_vars(bdt, 50))
    importance_dict[HNL_mass] = Sorted_importance(bdt)
    
elements_in_all = list(set.intersection(*map(set, list_of_lists)))
print(len(elements_in_all))
print(elements_in_all)

In [ ]:
highets_imps = ['n_showers', 'NeutrinoEnergy2', 'secondshower_Y_charge', 'trk_chipr_best', 'trk_energy_hits_tot', 'shr_energy_tot', 
                'contained_sps_ratio', 'pfnplanehits_Y', 'shrclusdir2', 'trk_dir_y_v', 'SliceCaloEnergy2', 'trk_theta_v', 'trk_start_x_v', 
                'pi0_radlen2', 'pfnplanehits_V', 'trk_energy', 'trk_score_v', 'shrclusdir0', 'trk_bragg_mip_v', 'shr_py_v', 
                'nu_flashmatch_score', 'n_pfps', 'CosmicIPAll3D', 'pi0_dir2_z']

ultimate_feature_list = ['n_pfps', 'n_tracks', 'shr_theta_v', 'shr_phi_v', 'shr_pz_v', 'shrclusdir2', 'shr_energy_tot',
                         'trk_theta_v', 'trk_phi_v','trk_dir_z_v', 'trk_energy', 'trk_energy_tot', 'trk_calo_energy_u_v', 'trk_score_v',
                         'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y', 'NeutrinoEnergy2', 'nu_flashmatch_score']

elements_in_all = list(set.intersection(*map(set, [highets_imps,ultimate_feature_list])))
print(len(elements_in_all))
print(elements_in_all)

In [ ]:
print(importance_dict[245].keys())
print(len(importance_dict[245].keys()))

In [ ]:
# for HNL_mass in Constants.HNL_mass_samples:
savefig = True
plt.figure(figsize=[20,8])
if Params["Load_pi0_signal"] == True: 
    colours = {"150_pi0":"coral", "200_pi0":"cornflowerblue", "245_pi0":"olivedrab"}
    smaller_samples = ["150_pi0", "200_pi0", "245_pi0"]
if Params["Load_pi0_signal"] == False: 
    colours = {"10_ee":"coral", "100_ee":"cornflowerblue", "150_ee":"olivedrab"}
    smaller_samples = ["10_ee", "100_ee", "150_ee"]
    
smaller_samples = ["10_ee", "100_ee", "150_ee"]#,"150_pi0", "200_pi0", "245_pi0"]
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

for i, HNL_mass in enumerate(smaller_samples):
    plt.bar(importance_dict[HNL_mass].keys(),importance_dict[HNL_mass].values(), label=f"{HNL_mass} model", 
            fill=False,linewidth=3, edgecolor=color_cycle[i])
    # plt.bar(importance_dict[HNL_mass].keys(),importance_dict[HNL_mass].values(), label=f"{HNL_mass}MeV model", 
    #         fill=False,linewidth=3, edgecolor=colours[HNL_mass], color=colours[HNL_mass])
# plt.bar(importance_dict[245].keys(),importance_dict[245].values())
plt.xticks(np.array(range(0, len(importance_dict[HNL_mass].keys()))),importance_dict[HNL_mass].keys(),rotation=80)
plt.ylabel("Importance")
plt.legend(fontsize=20)
plt.tight_layout()

if savefig==True:
    plt.savefig("plots/BDT_output/variable_importance/"+ Params["Run"]+f"_importances{BDT_name}.pdf")
    plt.savefig("plots/BDT_output/variable_importance/"+ Params["Run"]+f"_importances{BDT_name}.png")

In [ ]:
plt.figure(figsize=[20,8])
colours = {150:"coral", 200:"cornflowerblue", 245:"olivedrab"}
for HNL_mass in [245]:
    plt.bar(importance_dict[HNL_mass].keys(),importance_dict[HNL_mass].values(), label=f"{HNL_mass}MeV model", 
            fill=False,linewidth=3, edgecolor=colours[HNL_mass], color=colours[HNL_mass])
# plt.bar(importance_dict[245].keys(),importance_dict[245].values())
plt.xticks(np.array(range(0, len(importance_dict[HNL_mass].keys()))),importance_dict[HNL_mass].keys(),rotation=80)
plt.ylabel("Importance")
plt.legend(fontsize=20)
plt.tight_layout()

# plt.savefig("plots/BDT_output/variable_importance/"+ Params["Run"]+f"_importances_{BDT_name}.pdf")
# plt.savefig("plots/BDT_output/variable_importance/"+ Params["Run"]+f"_importances_{BDT_name}.png")

In [ ]:
fig = plt.figure(figsize=[20,20])
ax1 = fig.add_subplot(projection='3d')

mass_list = ["2MeV", "100MeV", "245MeV"]
x = importance_dict[2].keys()
y = mass_list
data = np.array([list(importance_dict[2].values()),
                 list(importance_dict[100].values()),
                 list(importance_dict[245].values())])

numOfCols = len(x)
numOfRows = len(y)

xpos = np.arange(0, numOfCols, 1)
ypos = np.arange(0, numOfRows, 1)
xpos, ypos = np.meshgrid(xpos + 0.5, ypos + 0.5)

xpos = xpos.flatten()
ypos = ypos.flatten()
zpos = np.zeros(numOfCols * numOfRows)

dx = np.ones(numOfRows * numOfCols) * 0.5
dy = np.ones(numOfCols * numOfRows) * 0.5
dz = data.flatten()

mass_list_labels = [" "," ", "2MeV"," ", "100MeV"," ", "245MeV"]

ax1.bar3d(xpos, ypos, zpos, dx, dy, dz)
ax1.set_xticklabels(list(importance_dict[2].keys()),rotation=45)
# ax1.set_xticklabels(list(importance_dict[2].keys()))
ax1.set_yticklabels(mass_list_labels)

ax1.set_xlabel('Variable')
ax1.set_ylabel('Model')
ax1.set_zlabel('Importance')
# plt.show()

# Finished code

In [ ]:
xgb_test_dict.keys()

In [ ]:
# dirt_matrix = xgboost.DMatrix(dirt_BDT[bdt_vars])
# EXT_matrix = xgboost.DMatrix(EXT_BDT[bdt_vars])

test_results_sig_dict = {}
test_results_bkg_dict = {}

train_results_sig_dict = {}
train_results_bkg_dict = {}

for HNL_mass in Constants.HNL_mass_samples:

    bdt = xgboost.Booster()
    bdt.load_model("bdts/"+Params["Run"]+f"_{HNL_mass}_MeV_New_20_variables_FIXED.json")
    #bdt.load_model(f'bdts/{Run}_{HNL_mass}_MeV_REDUCED_variables_flattened_highest_E_2.json')
    
    importance = bdt.get_score(importance_type="gain")
    
    for key in importance.keys():
        importance[key] = round(importance[key],1)
        
    #importance_dict[HNL_mass] = importance

    results_sig = bdt.predict(xgb_test_dict[HNL_mass])
    results_bkg = bdt.predict(xgb_test_bkg)
    
    train_results_sig = bdt.predict(xgb_sig_train_dict[HNL_mass])
    train_results_bkg = bdt.predict(xgb_bkg_train_dict[HNL_mass])
    
    test_results_sig_dict.update({HNL_mass:results_sig})
    test_results_bkg_dict.update({HNL_mass:results_bkg})
    
    train_results_sig_dict.update({HNL_mass:train_results_sig})
    train_results_bkg_dict.update({HNL_mass:train_results_bkg})

    # results_dirt = bdt.predict(dirt_matrix)
    # results_EXT = bdt.predict(EXT_matrix)
    
    # dirt_BDT[f'BDT_output_{HNL_mass}MeV'] = results_dirt
    # EXT_BDT[f'BDT_output_{HNL_mass}MeV'] = results_EXT

    # overlay_test_BDT[f'BDT_output_{HNL_mass}MeV'] = results_bkg
    # #Can add in a second loop over HNL_masses so that I predict each signal mass point with every other mass point bdt
    # signal_test_BDT_dict[HNL_mass][f'BDT_output'] = results_sig
    
    #Plotting importances of variables
    plt.figure(figsize=(12,12),facecolor='white')
    print(f"Plotting {HNL_mass}MeV importances:")
    a = xgboost.plot_importance(importance,max_num_features=10,importance_type='gain')
    

## Test vs. Train

In [ ]:
hist_range=[0,1.0]
n_bins=20

for HNL_mass in Constants.HNL_mass_samples:
    plt.figure(figsize=(10,7))
    plt.hist(train_results_sig_dict[HNL_mass],bins=n_bins, range=hist_range, density=True,alpha=0.4,color='red',label=f'Train {HNL_mass}MeV HNL' )
    counts,bin_edges = np.histogram(test_results_sig_dict[HNL_mass],bins=n_bins,range=hist_range,density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2.
    plt.plot(bin_centers,counts,marker='o',linestyle="None",color='red',label=f'Test {HNL_mass}MeV HNL')

    plt.hist(train_results_bkg_dict[HNL_mass], bins = n_bins, range = hist_range, density = True, alpha = 0.4, color = 'orange', label = r'Train overlay')
    counts,bin_edges = np.histogram(test_results_bkg_dict[HNL_mass],bins = n_bins, range= hist_range,density = True)
    bin_centers = (bin_edges[:-1] +  bin_edges[1:])/2.
    plt.plot(bin_centers,counts,marker='o',linestyle ="None",color='orange',label = r'Test overlay')
    plt.legend()

## Checking variable correlations

In [ ]:
#Taken from Luis' code
# for HNL_mass in HNL_masses:
method = 'kendall'
correlations = cleaned_signal_dict[100][bdt_vars].astype(np.float64).corr(method=method)
plt.figure(figsize=(15,12))
sns.heatmap(correlations,vmin=-1,annot=False,square=True,cbar_kws={'label':method+' correlation'},cmap = 'RdBu_r')
plt.title('Input Variable Correlations')
plt.show()

In [ ]:
#Just looking at most correlated 
corr=cleaned_signal_dict[100][bdt_vars].corr()
high_corr_var=np.where(corr>0.95)
high_corr_var=[(corr.columns[x],corr.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
high_corr_var

# End of code